<a href="https://colab.research.google.com/github/INT2-Team2/NeuralNetwork/blob/Adam/INT2_Group2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
%%capture
%%bash
pip install captum
pip install flask_compress
conda install freetype=2.10.4

In [28]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from captum.attr import IntegratedGradients
from captum.attr import LayerConductance
from captum.attr import NeuronConductance
from captum.insights import AttributionVisualizer, Batch
from captum.insights.attr_vis.features import ImageFeature
import time

In [29]:
if torch.cuda.is_available(): # use gpu if possible
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [30]:
epochs = 2
batch_size = 10
learning_rate = 0.0001

# only need resize so AlexNet works
transform = transforms.Compose([transforms.Resize((227,227)), 
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), 
                                                     (0.5, 0.5, 0.5))])

In [31]:
# get training/test data from CIFAR10 dataset
train_data = torchvision.datasets.CIFAR10(root = "./dataset", 
                                        train = True, 
                                        transform = transform, 
                                        download = True)
test_data = torchvision.datasets.CIFAR10(root = "./dataset", 
                                       train = False, 
                                       transform = transform, 
                                       download = True)

Files already downloaded and verified
Files already downloaded and verified


In [32]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, 
                                           batch_size = batch_size, 
                                           shuffle = True, 
                                           num_workers = 2)
test_loader = torch.utils.data.DataLoader(dataset = test_data, 
                                          batch_size = batch_size, 
                                          shuffle = False, 
                                          num_workers = 2)

In [33]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # Implementing AlexNet, try others later
    # Coppied a bit from:
    # https://medium.com/analytics-vidhya/alexnet-a-simple-implementation-using-pytorch-30c14e8b6db2
    # and
    # https://www.youtube.com/watch?v=pDdP0TFzsoQ&ab_channel=PythonEngineer

    #
    # Can't use AlexNet but can take 'inspiration' from it
    #
    # "You may take inspiration from existing network architectures, but you need to design/write the architecture yourself" (from the exam paper)
    #
    # need to change this section
    
    self.conv1 = nn.Conv2d(3, 96, 11, 4)
    self.conv2 = nn.Conv2d(96, 256, 5, 1, 2)
    self.conv3 = nn.Conv2d(256, 384, 3, 1, 1)
    self.conv4 = nn.Conv2d(384, 384, 3, 1, 1)
    self.conv5 = nn.Conv2d(384, 256, 3, 1, 1)

    self.maxPool = nn.MaxPool2d(3, 2)

    self.fc1 = nn.Linear(9216, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, 10)
  
  def forward(self, x):
    x = nn.functional.relu(self.conv1(x))
    x = self.maxPool(x)
    x = nn.functional.relu(self.conv2(x))
    x = self.maxPool(x)
    x = nn.functional.relu(self.conv3(x))
    x = nn.functional.relu(self.conv4(x))
    x = nn.functional.relu(self.conv5(x))
    x = self.maxPool(x)
    x = x.reshape(x.shape[0], -1)
    x = nn.functional.relu(self.fc1(x))
    x = nn.functional.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  

In [34]:
# Model creation

model = CNN().to(device)

In [35]:
def test():
  print("Testing")
  with torch.no_grad():
    correct = 0
    samples = 0

    for (images, labels) in test_loader:
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)
      _, predictions = outputs.max(1)

      samples += labels.size(0)
      correct += (predictions == labels).sum()

    print("Test accuracy was",100*float(correct)/float(samples))
    print()

In [36]:
## Training

# try other loss functions/optimizers
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

test_per_epoch = True
train_for_time = 20 # how many minutes to train for (and then finish current epoch)

if train_for_time:
  epochs = train_for_time*1000
start = time.time()

for epoch in range(epochs):
  print("Training")
  epoch_loss = 0
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # forwards
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backwards
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
  print("Epoch", epoch+1, "complete")
  print("Loss was", epoch_loss/len(train_loader))
  print()
  if test_per_epoch:
    test()
  
  if time.time()-start >= train_for_time*60:
    break

if not test_per_epoch:
  test()

Training
Epoch 1 complete
Loss was 1.3808172102332115

Testing
Test accuracy was 59.67

Epoch 2 complete
Loss was 0.8386848750084639

Testing
Test accuracy was 72.42

Epoch 3 complete
Loss was 0.6066096899449825

Testing
Test accuracy was 78.07



In [37]:
from google.colab import output
output.enable_custom_widget_manager()

In [38]:
def baseline_func(input):
  return 0

visualizer = AttributionVisualizer(
    models=[CNN],
    score_func=lambda o: torch.nn.functional.softmax(o, 1),
    classes= train_data.classes,
    features=[
        ImageFeature(
            "Photo",
            baseline_transforms=[baseline_func],
            input_transforms=[transform],
        )
    ],
    dataset=test_data,
)

In [39]:
visualizer.render()

CaptumInsights(insights_config={'classes': ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'h…

Output()